<a href="https://colab.research.google.com/github/minhthy1016/Credit-Innovators/blob/minhthy1016-patch-1/keyword_categorize_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow-text

     |████████████████████████████████| 3.4MB 5.6MB/s 


In [ ]:
pip install \
 apache-airflow==1.10.10 \
 --constraint \
        https://raw.githubusercontent.com/apache/airflow/1.10.10/requirements/requirements-python3.7.txt


     |████████████████████████████████| 4.7MB 4.1MB/s 
     |████████████████████████████████| 1.1MB 30.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 6.1MB 15.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 122kB 53.0MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 327kB 37.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |█████████████

In [ ]:
import tensorflow as tf
import timeit
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import os
from tqdm import tqdm 
import functools
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
import datetime
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators import gcs_to_bq
import csv
import json
from pprint import pprint


os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"

default_dag_args = {'ownder': 'your_email_here',
                    'depend_on_past': False, 
                    'start_date': days_ago(1),
                    'retries': 2,
                    'project_id': '<PROJECT_ID>'}

with DAG(dag_id='keyword_classify', description='keyword_classify',
         default_args=default_dag_args) as dag:
    ####Dummy Variable For Start
    START = DummyOperator(task_id='START')
    ####Dummy Variable For End
    FINISH = DummyOperator(task_id='FINISH', depends_on_past=True)
    # logging.error('table name: '+table_name)


preprocessor = hub.load("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1")

def query_competitor(country_code):
    #TODO: replace json path
    client = bigquery.Client.from_service_account_json(r'<credential-account-json-file-path>.json')
    query_text="""
    SELECT
    *
    FROM
  `<PROJECT_ID>.<DATASET_ID>.comp_by_cat_mf`
    WHERE
    date = (SELECT max(date) from `<PROJECT_ID.<DATASET_ID>.comp_by_cat_mf` where country_code='{}')
    AND country_code='{}'
    AND url not like'%hello%'
    """.format(country_code,country_code)
      #print(query_text)
    query_job = client.query(query_text)

    results = query_job.result().to_dataframe()
    return results

def query_cms(country_code):
    #TODO: replace json path
    client = bigquery.Client.from_service_account_json(r'<credential-account-json-file-path>.json')
    query_text="""
    SELECT
  category_1_en,
  category_2_en,
  post_title,
FROM (
  SELECT
    DISTINCT category_1_en,
    category_2_en,
    post_title,
    RANK() OVER (PARTITION BY post_id ORDER BY date DESC ) rank
  FROM
    `<PROJECT_ID.<DATASET_ID>.cms_df`
  WHERE
    category_1_en IS NOT NULL
    AND country_code='{}' )
WHERE
  rank=1
LIMIT 1000
    """.format(country_code)
      #print(query_text)
    query_job = client.query(query_text)
    results = query_job.result().to_dataframe()
    return results

def normalization(embeds):
  #norms = np.linalg.norm(embeds, 2, axis=1, keepdims=True)
    norms=tf.norm(embeds, 2, axis=1, keepdims=True)
    return embeds/norms

def embedding(batch_size=1000):
    def decor(func):
      @functools.wraps(func)
      def wrapper(tensor):
        gsc_embeds=func(tensor[:batch_size])
        #gsc_embeds=gsc_embeds.numpy()
        if tensor.shape[0]>batch_size:
          with tqdm(total=tensor.shape[0]//batch_size) as pbar:
            for i in range(tensor.shape[0]//batch_size):
                gsc_embeds2=func(tensor[batch_size*(i+1):batch_size*(i+2)])
                gsc_embeds=tf.concat([gsc_embeds, gsc_embeds2], 0)
                pbar.update(1)
        gsc_embeds=normalization(gsc_embeds)
        return gsc_embeds
      return wrapper
    return decor

@embedding(batch_size=1000)
def encoding(kw_tensor):
  #z = tf.matmul(x, y)
  return encoder(preprocessor(kw_tensor))["default"]

@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

def main():
  tw=query_competitor('TH')

  gsc=query_cms('ID')


  gsc_kw=[str(title) for title in list(gsc['post_title'])]
  tw_kw=[str(title) for title in list(tw['keyword'])]

  gsc_tensor = tf.constant(gsc_kw)
  tw_tensor  = tf.constant(tw_kw)


  gsc_embeds=encoding(gsc_tensor)
  tw_embeds= encoding(tw_tensor[:1000])

  sim=matmul_fn(tw_embeds, tf.transpose(gsc_embeds))
  sim_max=tf.math.reduce_max(sim, axis=-1)
  sim_indices=tf.math.argmax(sim, axis=-1).numpy()
  tw=tw.iloc[:1000,:]
  tw['sim_indices']=sim_indices
  tw['score']=sim_max.numpy()
  kw_map={i: v for i,v in zip(gsc.index, gsc['category_1_en'])}
  tw['hhg_cat_1']=tw['sim_indices'].map(kw_map)
  kw_map2={i: v for i,v in zip(gsc.index, gsc['category_2_en'])}
  tw['hhg_cat_2']=tw['sim_indices'].map(kw_map2)
 # tw[tw['score']>=0.65].to_csv('sample.csv', index=False, encoding='utf-8-sig')
  print(tw) 
run_keyword_classify_cmlm = PythonOperator(
    task_id='run_keyword_classify_cmlm',
    provide_context=False,
    depends_on_past=False,
    python_callable=main,
    dag=dag,
)

START >> run_keyword_classify_cmlm >> FINISH


<Task(DummyOperator): FINISH>